# 03 - Merton Jump Diffusion Model

## 0 - Setup Ambiente

In [6]:
import os, sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# Imposto la root del progetto a partire dalla posizione di questo file
PROJECT_ROOT = Path.cwd().parent
SRC_DIR = PROJECT_ROOT / "src"
DATA_RESULTS = PROJECT_ROOT / "data" / "results"
DATA_RESULTS.mkdir(parents=True, exist_ok=True)

# Aggiungo la cartella src al path per poter importare i moduli personalizzati
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

In [10]:
import importlib, merton_jd
importlib.reload(merton_jd) # per ricaricare il modulo dopo averlo modificato
from merton_jd import JDParams, simulate_merton_jd, price_euro_call_mc_jd, discounted_martingale_error, merton_cf_call
from bsm import bs_price, bs_greeks, implied_vol
from utils_plot import plot_paths

In [11]:
# Parametri base
S0, K, T, r, q = 100.0, 100.0, 1.0, 0.02, 0.00
params = JDParams(sigma=0.20, lam=0.60, mu_J=-0.10, sigma_J=0.25)

# 1) Consistenza con BSM quando lam=0
params_l0 = JDParams(sigma=0.20, lam=0.0, mu_J=-0.10, sigma_J=0.25)
cf_l0 = merton_cf_call(S0, K, T, r, q, params_l0, trunc=0)
bs = bs_price(S0, K, T, r, params_l0.sigma, q, kind="call")
print("CF λ=0:", cf_l0, " | BSM:", bs)
pr_mc, se_mc = price_euro_call_mc_jd(S0, K, r, q, params, T, N=252, M=20000, seed=7, antithetic=True)
pr_cf = merton_cf_call(S0, K, T, r, q, params, trunc=60)
print("Closed-form:", pr_cf)
print("Monte Carlo:", pr_mc, "±", 1.96*se_mc)


CF λ=0: 8.124489368981159  | BSM: 8.916037278572539
Closed-form: 9.039347827960746
Monte Carlo: 11.667349945675907 ± 0.2764567128434369
